# Making an example NEO4J db for the talk

In [62]:
import random

## Creating the nodes data

In [63]:
resource_codes = []
for i in range(0,5000):
    resource_codes.append("Res_" + str(i))

In [64]:
group_codes = []
for i in range(0,25000):
    group_codes.append("G_" + str(i))

In [65]:
person_codes = []
for i in range(0,150000):
    person_codes.append("P_" + str(i))

# Creating the edges randomly

In [66]:
person_is_in_group = []


# Assign every person to a group
for person in person_codes:
    person_is_in_group.append([person,random.choice(group_codes)])
    
# Assign a 50% of people to a random set of groups:

for person in random.sample(person_codes, int(round(len(person_codes)/2))):
    for extra_group in range(0,random.randint(1, 5)):
        person_is_in_group.append([person,random.choice(group_codes)])
    

In [67]:
group_child_of_group = []

for i in range(0, len(group_codes)):
    group_child_of_group.append([random.choice(group_codes),random.choice(group_codes)])

In [68]:
group_has_access_resource = []

for resource in resource_codes:
    group_has_access_resource.append([random.choice(group_codes),resource])


## Insert the data into Neo4j

In [69]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "q1w2"))

### Importing the nodes into NEO4J

In [70]:
with driver.session() as session:
    tx = session.begin_transaction()
    for resource in resource_codes:
        result = tx.run("CREATE (r:Resource {code:'"+resource+"'}) RETURN r.name")
    tx.commit()

In [71]:
with driver.session() as session:
    tx = session.begin_transaction()
    for group in group_codes:
        result = tx.run("CREATE (r:Group {code:'"+group+"'}) RETURN r.name")
    tx.commit()

In [ ]:
with driver.session() as session:
    tx = session.begin_transaction()
    for person in person_codes:
        result = tx.run("CREATE (r:Person {code:'"+person+"'}) RETURN r.name")
    tx.commit()

### Importing the edges into NEO4J

In [ ]:
# Match, Macth Create Rel